In [10]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from BsonIterator import BSONIterator

In [4]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.9_0.2.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.9_0.2.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 128

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock,
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock,
                       batch_size=batch_size, shuffle=True)

Found 988516 images belonging to 5270 classes.
Found 242274 images belonging to 5270 classes.


In [20]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# Save model Dir
MODEL_DIR = 'saved_models'

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions, name= 'InceptionV3')

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

num_epochs = 10

# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4)
model.save(os.path.join(MODEL_DIR, model.name + '_epcs' + str(num_epochs) + '_RandomSplit_Train_0.9_0.2' + '.h5'))

Epoch 1/10
7722/7722 [==============================] - ETA: 0s - loss: 4.2675 - acc: 0.3075 - val_loss: 3.8226 - val_acc: 0.3539
Epoch 2/10
7722/7722 [==============================] - ETA: 0s - loss: 3.6392 - acc: 0.3649 - val_loss: 3.7629 - val_acc: 0.3700
Epoch 3/10
7722/7722 [==============================] - ETA: 0s - loss: 3.4846 - acc: 0.3783 - val_loss: 3.8209 - val_acc: 0.3736
Epoch 4/10
7722/7722 [==============================] - ETA: 0s - loss: 3.4166 - acc: 0.3852 - val_loss: 3.8514 - val_acc: 0.3766
Epoch 5/10
7722/7722 [==============================] - ETA: 0s - loss: 3.3878 - acc: 0.3881 - val_loss: 3.8897 - val_acc: 0.3730
Epoch 6/10
7721/7722 [============================>.] - ETA: 0s - loss: 3.3835 - acc: 0.3893

KeyboardInterrupt: 

In [21]:
model.save(os.path.join(MODEL_DIR, model.name + '_epcs' + str(num_epochs) + '_RandomSplit_Train_0.9_0.2' + '.h5'))

In [23]:

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=["accuracy"])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4)

(0, 'input_2')
(1, 'conv2d_95')
(2, 'batch_normalization_95')
(3, 'activation_95')
(4, 'conv2d_96')
(5, 'batch_normalization_96')
(6, 'activation_96')
(7, 'conv2d_97')
(8, 'batch_normalization_97')
(9, 'activation_97')
(10, 'max_pooling2d_5')
(11, 'conv2d_98')
(12, 'batch_normalization_98')
(13, 'activation_98')
(14, 'conv2d_99')
(15, 'batch_normalization_99')
(16, 'activation_99')
(17, 'max_pooling2d_6')
(18, 'conv2d_103')
(19, 'batch_normalization_103')
(20, 'activation_103')
(21, 'conv2d_101')
(22, 'conv2d_104')
(23, 'batch_normalization_101')
(24, 'batch_normalization_104')
(25, 'activation_101')
(26, 'activation_104')
(27, 'average_pooling2d_10')
(28, 'conv2d_100')
(29, 'conv2d_102')
(30, 'conv2d_105')
(31, 'conv2d_106')
(32, 'batch_normalization_100')
(33, 'batch_normalization_102')
(34, 'batch_normalization_105')
(35, 'batch_normalization_106')
(36, 'activation_100')
(37, 'activation_102')
(38, 'activation_105')
(39, 'activation_106')
(40, 'mixed0')
(41, 'conv2d_110')
(42, 'batc

7722/7722 [==============================] - ETA: 0s - loss: 2.6654 - acc: 0.4701 - val_loss: 3.5815 - val_acc: 0.4251
Epoch 5/10
7722/7722 [==============================] - ETA: 0s - loss: 2.5934 - acc: 0.4794 - val_loss: 3.5646 - val_acc: 0.4295
Epoch 6/10
7721/7722 [============================>.] - ETA: 0s - loss: 2.5299 - acc: 0.4879

KeyboardInterrupt: 